# MUTRITION RECOMMENDER SYSTEM

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

usuariosDF = pd.read_csv('userPreferences.csv')
usuariosDF.drop('Unnamed: 0', axis=1, inplace=True)

recetasDF = pd.read_csv('Recetas_procesadas.csv')
recetasDF = recetasDF[['title', 'calories', 'protein', 'fat', 'sodium']]

In [2]:
dfmixta = usuariosDF.merge(recetasDF, on='title')

Buscar los 50 usuarios más parecidos a ti. Crear una tuple (index, similarity_score), luego ordenamos los ususarios por el similarity_score y nos quedamos solo con lso primeros 50.

In [3]:
userinfo={"peso":[80], "altura":  [160]}
dt=pd.DataFrame(userinfo)
user_sim_score=cosine_similarity(dt, usuariosDF[['peso', 'altura']])
user_sim_score=user_sim_score[0]

users_sim_score_tuple: List[tuple] = []
i = 0
for x in user_sim_score:
    users_sim_score_tuple.append((i, x))
    i+=1

sorted_by_score = sorted(users_sim_score_tuple, key=lambda tup: tup[1], reverse=True)
top_50 = sorted_by_score[:50]
#user_sim_score[::-1].sort()

# Buscar las receteas que han recomendado esos usuarios.
En la lista top_50, tenemos una tuple(index, sim_score), ahora crearemos un dataset que vincule basandose en el index la lista top_50 con la dfmixta que es la que contiene las recetas que ha recomendado cada usuario. 

In [4]:
recomendationDF = pd.DataFrame()
for i in top_50:
    recomendationDF = recomendationDF.append(dfmixta.iloc[i[0]])
recomendationDF.head(5)

,peso,altura,title,calories,protein,fat,sodium
27,83,151,Stuffed Sweet Potatoes,254.0,6.0,13.0,486.0
55,57,177,Roasted Bell Pepper Salad with Pine Nuts,96.0,2.0,7.0,5.0
1131,94,199,Chocolate-Cherry Ice Cream Pie with Hot Fudge ...,658.0,7.0,39.0,209.0
1552,68,186,Portobello Mushrooms Stuffed with Parmesan Whi...,297.0,13.0,13.0,309.0
1591,97,197,Raspberry Sorbet,246.0,1.0,1.0,2.0


Codificar el texto para que encuentre similitudes

In [5]:
#Importing TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Defining a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Constructing the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(recomendationDF['title'])

#Output the shape of tfidf_matrix
tfidf_matrix = tfidf.fit_transform(recomendationDF['title'])

tfidf_matrix.shape

(50, 160)

Calcular la matriz de cosine similarity de las 50 ceteas con la intencion de recomendar la receta que se asimila más con las demas.

In [6]:
# Importing linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Computing the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

cosine_sim[2] # cosine similarity de la receta [2] comparada con el resto.

array([0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.13672099, 0.13672099, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.08750765, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.09643385, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.13310429, 0.28713219, 0.        , 0.        , 0.15048165])

In [7]:
cosine_sim[2][0] # receta [2] comparada con la [0]

0.0

Recorrer la matriz y sumar todos los valores, la receta que tenga la mayor suma de cosine similarity sera la que se recomiende.

In [8]:
indexLaMejor = 0
mejorResultado = 0.0
actual = 0.0

for i in range (0, 50):
    for j in range(0, 50):
        actual+=cosine_sim[i,j]
    if actual > mejorResultado:
        mejorResultado=actual
        indexLaMejor=i
        actual=0
    else:
        actual=0.0
print(indexLaMejor)

22


In [23]:
similar_food = list(enumerate(cosine_sim[indexLaMejor]))
sorted_similar_food = sorted(similar_food, key=lambda x:x[1], reverse=True)
segundaRecetaIndex = sorted_similar_food[1][0]
segundaRecetaIndex

30

Ahora recoger la receta buscando por el index en el datframe de recomendaciones (que contiene la recomendacion de los top 50 usuarios mas parecidos a ti)

In [16]:
receta_recomendar=recomendationDF[indexLaMejor:indexLaMejor+1]
recetaNP=receta_recomendar.to_numpy() #conertirlo a numpy
recetaList = recetaNP.tolist() #pasarlo a lista

In [26]:
segunda=recomendationDF[segundaRecetaIndex:segundaRecetaIndex+1].to_numpy().tolist()
recetaList.append(segunda)
print(recetaList)

[[68, 165, 'Grilled Lemon Chicken Salad with Dill Cream Dressing ', 617.0, 49.0, 37.0, 307.0], [[87, 192, 'Cream Cheese-Lemon Zest Frosting ', 293.0, 2.0, 23.0, 140.0]]]
